In [2]:
#importing relevant libraries
import requests
import pandas as pd

In [1]:
# setting url to yelp API.
# adding paramters to search for restaurants in Denver.
# adding personal yelp API key. NOTE: need to edit to add your personal key before running code.

url = "https://api.yelp.com/v3/businesses/search"

term = "Restaurants"
latitude = 39.752660 #Lat/ long are for union station
longitude = -104.999967
radius = 3200  #search radius from union station in meters. roughly 2 miles. Yelp uses as a suggestion; actual radius may vary.
params = dict(term = term, latitude = latitude, longitude = longitude, radius = radius)

api_key = 'REDACTED' # edit before running code
headers = dict(Authorization = "Bearer "+api_key)

In [3]:
# checking the total number of search results. Can only pull up to 1000 with yelp
response = requests.get(url = url, params = params, headers=headers)
response_dict = response.json()
response_dict['total']

919

In [4]:
# requesting up to 50 restaurants at a time (yelp max) up to total number of search results.

num_loops = response_dict['total'] // 50 + 1
last_loop_limit = response_dict['total'] % 50
offset = 0
limit = 50
restaurants = []

for i in range(num_loops):
    if i == max(range(num_loops)):
        limit = last_loop_limit
    params = dict(term = term, latitude = latitude, longitude = longitude, radius = radius, limit = limit, offset = offset)
    response = requests.get(url = url, params = params, headers=headers)
    response_dict = response.json()
    restaurants.extend(response_dict['businesses'])
    offset += 50

In [5]:
df = pd.DataFrame(restaurants)

In [6]:
#putting location information into its own columns and dropping the location dictionary
df['address1'] = df['location'].apply(lambda loc_dict: loc_dict['address1'])
df['address2'] = df['location'].apply(lambda loc_dict: loc_dict['address2'])
df['address3'] = df['location'].apply(lambda loc_dict: loc_dict['address3'])
df['city'] = df['location'].apply(lambda loc_dict: loc_dict['city'])
df['country'] = df['location'].apply(lambda loc_dict: loc_dict['country'])
df['display_address'] = df['location'].apply(lambda loc_dict: loc_dict['display_address'])
df['state'] = df['location'].apply(lambda loc_dict: loc_dict['state'])
df['zip_code'] = df['location'].apply(lambda loc_dict: loc_dict['zip_code'])
df.drop(['location'], axis = 1, inplace=True)

In [7]:
#putting the latitude and longitude into their own columns and deleting the coordinates dictionary
df['latitude'] = df['coordinates'].apply(lambda coord_dict: coord_dict['latitude'])
df['longitude'] = df['coordinates'].apply(lambda coord_dict: coord_dict['longitude'])
df.drop(['coordinates'], axis =1,  inplace=True)

In [9]:
# There are up to three categories per restaurant. Added to their own columns and dropped the categories dictionary
df['categories_num'] = df['categories'].apply(lambda cat_list: len(cat_list))
df['category_1_alias'] = df['categories'].apply(lambda cat_list: cat_list[0]['alias'] if len(cat_list) > 0 else None)
df['category_1_title'] = df['categories'].apply(lambda cat_list: cat_list[0]['title'] if len(cat_list) > 0 else None)
df['category_2_alias'] = df['categories'].apply(lambda cat_list: cat_list[1]['alias'] if len(cat_list) > 1 else None)
df['category_2_title'] = df['categories'].apply(lambda cat_list: cat_list[1]['title'] if len(cat_list) > 1 else None)
df['category_3_alias'] = df['categories'].apply(lambda cat_list: cat_list[2]['alias'] if len(cat_list) > 2 else None)
df['category_3_title'] = df['categories'].apply(lambda cat_list: cat_list[2]['title'] if len(cat_list) > 2 else None)
df.drop(['categories'], axis =1, inplace=True)

In [11]:
df.to_csv(path_or_buf=r'C:\Users\MainUser\Desktop\Restaurants.csv', index=False)